In [15]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')

In [16]:
venues = venues.dropna()

In [17]:
import folium

# Create a map centered at the average latitude and longitude
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m)


# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
# Add markers for each bus terminal
for _, row in bus_terminals.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='green')
    ).add_to(m)
# Display the map
m

In [18]:
demand= accomodations_clusters['total_accommodates'].to_numpy()
demand

array([ 6969,  6395,  2801,  7026,  4909,  1545,  6342,  6399,  1171,
        4117,  3110,  3088,  5080,  4779,  2666,  9095,  7555,  4679,
        2111,  3376,  7538,  7068,  6853,  1576,  2974,  2110,  1319,
       10383,  1268,   273,  8063,  8239,  3833,  6408,  7028,  2057,
        4099,  2060,  2646,  2505,  6751,   934,  4119,  9229,   563,
        7296,  2670,  2156,  2263,  1619, 12812])

In [19]:
from ortools.sat.python import cp_model

# Set the number of bus stops to locate
p = 15

# bus_matrix rows: accommodations; columns: candidate bus terminals
accom = list(bus_matrix.index)
bus_stops = list(bus_matrix.columns[1:])

model = cp_model.CpModel()

# Decision variables:
x = {j: model.NewBoolVar(f"x_{j}") for j in bus_stops}
y = {}
for idx, i in enumerate(accom):
    for j in bus_stops:
        y[(idx, j)] = model.NewBoolVar(f"y_{idx}_{j}")

# Constraint: each accommodation must be assigned to exactly one bus stop.
for idx, i in enumerate(accom):
    model.Add(sum(y[(idx, j)] for j in bus_stops) == 1)

# Constraint: assignment only possible if bus stop is selected.
for idx, i in enumerate(accom):
    for j in bus_stops:
        model.Add(y[(idx, j)] <= x[j])

# Constraint: exactly p bus stops are selected.
model.Add(sum(x[j] for j in bus_stops) == p)

# Objective: minimize total (demand-weighted) distance.
# We convert bus_matrix.loc[i, j] to an integer cost.
objective_terms = []
for idx, i in enumerate(accom):
    for j in bus_stops:
        # Multiply demand and the scaled distance.
        cost = int(round(bus_matrix.loc[i, j]))
        objective_terms.append(demand[idx] * cost * y[(idx, j)])
model.Minimize(sum(objective_terms))

# Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    selected = [j for j in bus_stops if solver.Value(x[j]) == 1]
    print("Selected bus stops:", selected)
else:
    print("No solution found.")

Selected bus stops: ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']


In [20]:
#map only selected bus stops ['BD14', 'BD15', 'BT03', 'BL14', 'BT23', 'BT18', 'BL23', 'BL07', 'BT05']
# Add markers for each cluster
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m2 = folium.Map(location=map_center, zoom_start=10)

for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m2)
# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m2)

for _, row in bus_terminals[bus_terminals['id'].isin(selected)].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='lightgreen', icon='ok-sign')
    ).add_to(m2)
# Display the map with selected bus stops
m2

In [21]:
merged_matrix

,Unnamed: 0,A1,A10,A11,A12,A13,A14,A15,A16,A17,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
0,A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
1,A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
2,A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
3,A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
4,A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
150,BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,...,1126.0,1238.0,1647.0,840.0,857.0,952.0,0.0,1244.0,1479.0,914.0
151,BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,...,933.0,1044.0,1418.0,1854.0,2018.0,2087.0,1345.0,0.0,357.0,707.0
152,BT27,1485.0,2420.0,3408.0,1424.0,1579.0,1874.0,1556.0,1447.0,2584.0,...,1164.0,1311.0,1731.0,1914.0,1635.0,2047.0,1405.0,362.0,0.0,987.0


In [22]:
# Check if merged_matrix is a square matrix
if merged_matrix.shape[0] == merged_matrix.shape[1]:
    print("merged_matrix is a square matrix.")
else:
    print("merged_matrix is not a square matrix.")

merged_matrix is not a square matrix.


In [23]:
print(merged_matrix.to_string())

    Unnamed: 0       A1      A10      A11      A12      A13      A14      A15      A16      A17      A18      A19       A2      A20      A21      A22      A23      A24      A25      A26      A27      A28      A29       A3      A30      A31      A32      A33      A34      A35      A36      A37      A38      A39       A4      A40      A41      A42      A43      A44      A45      A46      A47      A48      A49       A5      A50      A51       A6       A7       A8       A9       V1      V10      V11      V12      V13      V14      V15      V16      V17      V18      V19      V2      V20      V21      V22      V23      V24      V25      V26      V27      V28      V29       V3      V30      V31      V32      V33  V34       V4       V5       V6       V7       V8       V9     BD01     BD02     BD03     BD04     BD05     BD06     BD07     BD08     BD09     BD10     BD11     BD12     BD13     BD14     BD15     BD16     BD17     BL01     BL02     BL03     BL04     BL05     BL06     BL07     BL08 

In [24]:
# Create a list of pickup and delivery pairs to match accommodations (pickup) and venues (delivery)
pickup_delivery_pairs = [(accommodation, venue) for accommodation in accomodations_clusters['id'] for venue in venues['id']]

# Display the list of pairs
print("Pickup and Delivery Pairs:")
print(pickup_delivery_pairs)

Pickup and Delivery Pairs:
[('A1', 'V1'), ('A1', 'V2'), ('A1', 'V3'), ('A1', 'V4'), ('A1', 'V5'), ('A1', 'V6'), ('A1', 'V7'), ('A1', 'V8'), ('A1', 'V9'), ('A1', 'V10'), ('A1', 'V11'), ('A1', 'V12'), ('A1', 'V13'), ('A1', 'V14'), ('A1', 'V15'), ('A1', 'V16'), ('A1', 'V17'), ('A1', 'V18'), ('A1', 'V19'), ('A1', 'V20'), ('A1', 'V21'), ('A1', 'V22'), ('A1', 'V23'), ('A1', 'V24'), ('A1', 'V25'), ('A1', 'V26'), ('A1', 'V27'), ('A1', 'V28'), ('A1', 'V29'), ('A1', 'V30'), ('A1', 'V31'), ('A1', 'V32'), ('A1', 'V33'), ('A2', 'V1'), ('A2', 'V2'), ('A2', 'V3'), ('A2', 'V4'), ('A2', 'V5'), ('A2', 'V6'), ('A2', 'V7'), ('A2', 'V8'), ('A2', 'V9'), ('A2', 'V10'), ('A2', 'V11'), ('A2', 'V12'), ('A2', 'V13'), ('A2', 'V14'), ('A2', 'V15'), ('A2', 'V16'), ('A2', 'V17'), ('A2', 'V18'), ('A2', 'V19'), ('A2', 'V20'), ('A2', 'V21'), ('A2', 'V22'), ('A2', 'V23'), ('A2', 'V24'), ('A2', 'V25'), ('A2', 'V26'), ('A2', 'V27'), ('A2', 'V28'), ('A2', 'V29'), ('A2', 'V30'), ('A2', 'V31'), ('A2', 'V32'), ('A2', 'V33'), 

#MOCK VRP

In [30]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),
    data['num_vehicles'],
    data['depot_indices'],  # Start nodes
    data['depot_indices'],  # End nodes
)

routing = pywrapcp.RoutingModel(manager)


#convert matrices to integer format

In [56]:
merged_matrix_copy = merged_matrix.copy()

# Create the mapping of location names to row indices
location_to_index = {name.strip().lower(): idx for idx, name in enumerate(merged_matrix_copy['Unnamed: 0'])}

# Debugging: Print the mapping for a few examples
print(location_to_index)  # Check a few location-to-index mappings


{'a1': 0, 'a10': 1, 'a11': 2, 'a12': 3, 'a13': 4, 'a14': 5, 'a15': 6, 'a16': 7, 'a17': 8, 'a18': 9, 'a19': 10, 'a2': 11, 'a20': 12, 'a21': 13, 'a22': 14, 'a23': 15, 'a24': 16, 'a25': 17, 'a26': 18, 'a27': 19, 'a28': 20, 'a29': 21, 'a3': 22, 'a30': 23, 'a31': 24, 'a32': 25, 'a33': 26, 'a34': 27, 'a35': 28, 'a36': 29, 'a37': 30, 'a38': 31, 'a39': 32, 'a4': 33, 'a40': 34, 'a41': 35, 'a42': 36, 'a43': 37, 'a44': 38, 'a45': 39, 'a46': 40, 'a47': 41, 'a48': 42, 'a49': 43, 'a5': 44, 'a50': 45, 'a51': 46, 'a6': 47, 'a7': 48, 'a8': 49, 'a9': 50, 'v1': 51, 'v10': 52, 'v11': 53, 'v12': 54, 'v13': 55, 'v14': 56, 'v15': 57, 'v16': 58, 'v17': 59, 'v18': 60, 'v19': 61, 'v2': 62, 'v20': 63, 'v21': 64, 'v22': 65, 'v23': 66, 'v24': 67, 'v25': 68, 'v26': 69, 'v27': 70, 'v28': 71, 'v29': 72, 'v3': 73, 'v30': 74, 'v31': 75, 'v32': 76, 'v33': 77, 'v34': 78, 'v4': 79, 'v5': 80, 'v6': 81, 'v7': 82, 'v8': 83, 'v9': 84, 'bd01': 85, 'bd02': 86, 'bd03': 87, 'bd04': 88, 'bd05': 89, 'bd06': 90, 'bd07': 91, 'bd08': 

# VRP

In [34]:
! pip install ortools
from ortools.sat.python import cp_model
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [57]:
def create_data_model():
    data = {}
    data['distance_matrix'] = merged_matrix.values.tolist()  # Convert DataFrame to list of lists

    # Create a copy of merged_matrix to avoid modifying the original one
    merged_matrix_copy = merged_matrix.copy()

    # Map location names to indices
    location_to_index = {name.strip().lower(): idx for idx, name in enumerate(merged_matrix_copy['Unnamed: 0'])}

    # Debugging: Check the location-to-index mapping
    print(location_to_index)

    # Convert pickup/delivery names to integer indices using the location_to_index map
    data['pickups_deliveries'] = [
        [location_to_index[pickup.strip().lower()], location_to_index[delivery.strip().lower()]]
        for pickup, delivery in pickup_delivery_pairs
    ]

    # Debugging: Check the first 5 converted pairs
    print(data['pickups_deliveries'][:5])

    data['starts'] = list(range(126, 142))  # Assuming your bus starts are rows 126 to 141
    data['ends'] = list(range(126, 142))    # Same for ends (if applicable)

    # If needed, map venue names to indices as well
    data['venues'] = {location_to_index[v.strip().lower()] for v in venues['id']}  # For venue IDs, convert to indices
    
    return data

# Run the function to create the data model
data = create_data_model()

# Check the converted pickup/delivery pairs
print(data['pickups_deliveries'][:5])  # First 5 pickup/delivery pairs in integer indices


{'a1': 0, 'a10': 1, 'a11': 2, 'a12': 3, 'a13': 4, 'a14': 5, 'a15': 6, 'a16': 7, 'a17': 8, 'a18': 9, 'a19': 10, 'a2': 11, 'a20': 12, 'a21': 13, 'a22': 14, 'a23': 15, 'a24': 16, 'a25': 17, 'a26': 18, 'a27': 19, 'a28': 20, 'a29': 21, 'a3': 22, 'a30': 23, 'a31': 24, 'a32': 25, 'a33': 26, 'a34': 27, 'a35': 28, 'a36': 29, 'a37': 30, 'a38': 31, 'a39': 32, 'a4': 33, 'a40': 34, 'a41': 35, 'a42': 36, 'a43': 37, 'a44': 38, 'a45': 39, 'a46': 40, 'a47': 41, 'a48': 42, 'a49': 43, 'a5': 44, 'a50': 45, 'a51': 46, 'a6': 47, 'a7': 48, 'a8': 49, 'a9': 50, 'v1': 51, 'v10': 52, 'v11': 53, 'v12': 54, 'v13': 55, 'v14': 56, 'v15': 57, 'v16': 58, 'v17': 59, 'v18': 60, 'v19': 61, 'v2': 62, 'v20': 63, 'v21': 64, 'v22': 65, 'v23': 66, 'v24': 67, 'v25': 68, 'v26': 69, 'v27': 70, 'v28': 71, 'v29': 72, 'v3': 73, 'v30': 74, 'v31': 75, 'v32': 76, 'v33': 77, 'v34': 78, 'v4': 79, 'v5': 80, 'v6': 81, 'v7': 82, 'v8': 83, 'v9': 84, 'bd01': 85, 'bd02': 86, 'bd03': 87, 'bd04': 88, 'bd05': 89, 'bd06': 90, 'bd07': 91, 'bd08': 

In [58]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [59]:
print(data['pickups_deliveries'][:5])
print(type(data['pickups_deliveries'][0][0]), type(data['pickups_deliveries'][0][1]))


[[0, 51], [0, 62], [0, 73], [0, 79], [0, 80]]
<class 'int'> <class 'int'>


In [60]:
for pickup, delivery in data['pickups_deliveries']:
    pickup_index = manager.NodeToIndex(pickup)
    delivery_index = manager.NodeToIndex(delivery)
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
    routing.solver().Add(routing.CumulVar(pickup_index, 'Distance') <= routing.CumulVar(delivery_index, 'Distance'))


AttributeError: 'RoutingModel' object has no attribute 'CumulVar'

In [ ]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.time_limit.seconds = 10

solution = routing.SolveWithParameters(search_parameters)


In [ ]:
print("Distance matrix size:", len(merged_matrix))
print("Max start/end index:", max(data['starts'] + data['ends']))

Distance matrix size: 154
Max start/end index: 141


In [ ]:
#create routing index manager

#manager = pywrapcp.RoutingIndexManager(
    #data['num_vehicles'],
    #data['starts'],
    #data['ends']
)

##routing = pywrapcp.RoutingModel(manager)

: 

In [ ]:
#define distance function
#def distance_callback(from_index, to_index):
    #from_node = manager.IndexToNode(from_index)
    #to_node = manager.IndexToNode(to_index)
    #return data['distance_matrix'][from_node][to_node]

#transit_callback_index = routing.RegisterTransitCallback(distance_callback)
#routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


NameError: name 'routing' is not defined

In [ ]:
# Add constraints for pickup and delivery
#for pickup, delivery in data['pickups_deliveries']:
    # Convert pickup and delivery to indices using the manager
    #pickup_index = manager.NodeToIndex(accomodations_clusters[accomodations_clusters['id'] == pickup].index[0])
    #delivery_index = manager.NodeToIndex(venues[venues['id'] == delivery].index[0])
    
    #routing.AddPickupAndDelivery(pickup_index, delivery_index)
    #routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))

In [ ]:
#solve routing problem
#search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#search_parameters.first_solution_strategy = (
    #routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

##solution = routing.SolveWithParameters(search_parameters)


In [ ]:
#print solution
#def print_solution(data, manager, routing, solution):
    #total_distance = 0
    #for vehicle_id in range(data['num_vehicles']):
        #index = routing.Start(vehicle_id)
        #route_distance = 0
        #plan_output = f'Route for vehicle {vehicle_id}:\n'
        #while not routing.IsEnd(index):
            #node = manager.IndexToNode(index)
            #plan_output += f' {node} ->'
            #previous_index = index
            #index = solution.Value(routing.NextVar(index))
            #route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        #node = manager.IndexToNode(index)
        #plan_output += f' {node}\nDistance of the route: {route_distance}m\n'
        #print(plan_output)
        #total_distance += route_distance
    #print(f'Total distance of all routes: {total_distance}m')

#if solution:
    #print_solution(data, manager, routing, solution)
#else:
    #print("No solution found.")


No solution found.
